In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras import layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

Data Loading and Preparation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './Train/Train_aug',
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    './Val/Val_aug',
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

Model Construction and Compilation

In [ ]:
def create_model(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

    for layer in base_model.layers:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

Hyperparameter Tuning with Cross-Validation

In [ ]:
param_grid = {
    'optimizer': [tf.keras.optimizers.Adam(learning_rate=0.0001),
                  tf.keras.optimizers.RMSprop(learning_rate=0.0001)]
}

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=StratifiedKFold(n_splits=3))
grid_result = grid.fit(train_generator, steps_per_epoch=len(train_generator), validation_data=validation_generator, validation_steps=len(validation_generator))

print("Best Optimizer:", grid_result.best_params_['optimizer'])

model = create_model(optimizer=grid_result.best_params_['optimizer'])
model.summary()

Model Training

In [ ]:
history = model.fit(train_generator,
                    epochs=20,
                    validation_data=validation_generator)

model.save('ad_creative_detection_model.h5')

Feature Visualization Integration

In [ ]:
# Feature Visualization Integration
def analyze_features(model, img_path, layer_names):
    # Load and preprocess the image
    img = plt.imread(img_path)
    img = tf.image.resize(img, [200, 200])
    img_array = np.expand_dims(img, axis=0)
    img_array = img_array / 255.0

    for layer_name in layer_names:
        intermediate_layer_model = models.Model(inputs=model.input,
                                                outputs=model.get_layer(layer_name).output)
        intermediate_output = intermediate_layer_model.predict(img_array)

Advanced False Positive Reduction Exploration

In [ ]:
class AttentionLayer(layers.Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        x = tf.matmul(inputs, self.W)
        x = tf.nn.tanh(x + self.b)
        x = tf.nn.softmax(x, axis=1)
        return x * inputs

# Modify model to incorporate attention mechanism
attention_model = models.Sequential([
    base_model,
    AttentionLayer(),
    layers.Flatten(name='flatten'),
    layers.Dense(512, activation='relu', name='dense'),
    layers.Dense(1, activation='sigmoid', name='dense_1')
])

# Compile and train the attention model
attention_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

attention_model.summary()

history_attention = attention_model.fit(train_generator,
                                        epochs=20,
                                        validation_data=validation_generator)

Evaluation

In [ ]:
def evaluate_model(model, generator):
    y_true = generator.classes
    y_pred = model.predict(generator)
    y_pred = np.where(y_pred > 0.5, 1, 0)

    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, target_names=generator.class_indices.keys())

    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)

evaluate_model(model, validation_generator)

 Plotting and Final Validation Accuracy

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

final_val_accuracy = history.history['val_accuracy'][-1]
print("Final Validation Accuracy:", final_val_accuracy)


Test Prediction

In [ ]:
def predict_ad_creative(model, img_path):
    test_img = Image.open(img_path)
    test_img = test_img.resize((200, 200))  # Resize to match model input shape
    test_img = np.array(test_img) / 255.0  # Normalize pixel values

    # Perform prediction on the test image
    prediction = model.predict(np.expand_dims(test_img, axis=0))

    # Interpret prediction
    class_labels = ['Non-Ad', 'Ad']
    predicted_class = class_labels[int(np.round(prediction[0][0]))]
    return predicted_class

# Test the model with a sample image
test_image_path = 'test_image.jpg'
predicted_class = predict_ad_creative(model, test_image_path)
print("Predicted Class:", predicted_class)

FINAL CODE

import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras import layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Data Loading and Preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './Train/Train_aug',
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    './Val/Val_aug',
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary'
)

# Model Construction and Compilation
def create_model(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

    for layer in base_model.layers:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Hyperparameter Tuning with Cross-Validation
param_grid = {
    'optimizer': [tf.keras.optimizers.Adam(learning_rate=0.0001),
                  tf.keras.optimizers.RMSprop(learning_rate=0.0001)]
}

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=StratifiedKFold(n_splits=3))
grid_result = grid.fit(train_generator, steps_per_epoch=len(train_generator), validation_data=validation_generator, validation_steps=len(validation_generator))

print("Best Optimizer:", grid_result.best_params_['optimizer'])

model = create_model(optimizer=grid_result.best_params_['optimizer'])
model.summary()

# Model Training
history = model.fit(train_generator,
                    epochs=20,
                    validation_data=validation_generator)

model.save('ad_creative_detection_model.h5')

# Feature Extraction and Analysis
def extract_features(model, img_path):
    img = plt.imread(img_path)
    img = tf.image.resize(img, [200, 200])
    img_array = np.expand_dims(img, axis=0)
    img_array = img_array / 255.0

    intermediate_layer_model = models.Model(inputs=model.input,
                                            outputs=model.get_layer('global_average_pooling2d').output)
    features = intermediate_layer_model.predict(img_array)
    return features

# False Positive Reduction with Attention Mechanism
class AttentionLayer(layers.Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        x = tf.matmul(inputs, self.W)
        x = tf.nn.tanh(x + self.b)
        x = tf.nn.softmax(x, axis=1)
        return x * inputs

# Modify model to incorporate attention mechanism
attention_model = models.Sequential([
    base_model,
    AttentionLayer(),
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile and train the attention model
attention_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

attention_model.summary()

history_attention = attention_model.fit(train_generator,
                                        epochs=20,
                                        validation_data=validation_generator)

# Evaluation
def evaluate_model(model, generator):
    y_true = generator.classes
    y_pred = model.predict(generator)
    y_pred = np.where(y_pred > 0.5, 1, 0)

    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, target_names=generator.class_indices.keys())

    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(cr)

evaluate_model(model, validation_generator)

# Plotting and Final Validation Accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

final_val_accuracy = history.history['val_accuracy'][-1]
print("Final Validation Accuracy:", final_val_accuracy)

# Test Prediction
def predict_ad_creative(model, img_path):
    test_img = Image.open(img_path)
    test_img = test_img.resize((200, 200))  # Resize to match model input shape
    test_img = np.array(test_img) / 255.0  # Normalize pixel values

    # Perform prediction on the test image
    prediction = model.predict(np.expand_dims(test_img, axis=0))

    # Interpret prediction
    class_labels = ['Non-Ad', 'Ad']
    predicted_class = class_labels[int(np.round(prediction[0][0]))]
    return predicted_class

# Test the model with a sample image
test_image_path = 'test_image.jpg'
predicted_class = predict_ad_creative(model, test_image_path)
print("Predicted Class:", predicted_class)
